In [1]:
import contextlib
import os
import re
import sys

import linearmodels as lm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns
import tabulate

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [6]:
sample = "X77"
fn = os.path.join("s3://3di-data-mdb/raw", f"mdb_{sample}.parquet")
raw = ha.read_parquet(fn)
clean = md.clean_data(raw)
hd.inspect(clean)

(6,765,276, 28)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym
4328,1212603,2012-07-30,77,40.0,lloyds bank 28jul,NaN,spend,other_spend,0.0,kt3 5,2010-07-10,NaN,1945.0,2012-10-26,259583,2015-09-11,natwest bank,current,2014-07-18,2017-10-24,True,NaN,personal,cash,NaN,cash,u,201207
4330,1212608,2012-07-30,77,10.0,mdbremoved,NaN,NaN,NaN,0.0,kt3 5,2010-07-10,NaN,1945.0,2012-10-26,259583,2015-09-11,natwest bank,current,2014-07-18,2017-10-23,True,NaN,non merchant mbl,NaN,NaN,NaN,u,201207


In [ ]:
def tester(df):
    g = df.groupby("user_id")
    latest_balances_available = g.latest_balance.min().notna()
    valid_last_refresh_dates = g.account_last_refreshed.min() >= g.date.min()
    cond = latest_balances_available & valid_last_refresh_dates
    users = cond[cond].index
    return df[df.user_id.isin(users)]

In [120]:
data = (
    clean.pipe(sl.min_number_of_months)
    .pipe(sl.no_missing_months)
    .pipe(sl.account_balances_available)
)

In [121]:
data.user_id.nunique(), sl.no_missing_months(data).user_id.nunique()

(1652, 1652)

In [114]:
df = hd.read_sample("X77")

Time for read_sample    : 28.53 seconds


In [115]:
df.user_id.nunique(), sl.no_missing_months(df).user_id.nunique()

(599, 595)

In [124]:
import entropy.data.txn_classifications as tc

tc.tag_groups.keys()

dict_keys(['income', 'spend', 'transfers'])

In [136]:
data.tag_group.cat.categories

Index(['income', 'spend', 'transfers'], dtype='object')

In [140]:
spend = data[data.tag_group.eq('spend')]
spend.tag.unique()

['hobbies', 'finance', 'communication', 'household', 'services', 'motor', 'other_spend', 'retail', 'travel']
Categories (15, object): ['benefits', 'communication', 'earnings', 'finance', ..., 'savings', 'services', 'transfers', 'travel']